In [1]:
# 导入必需的库
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt

# 由于我们安装的是 tensorflow 2.X，要使用 1.X 的代码，需要使用下列语句导入
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

# 如果安装的是 1.X，则使用下列语句导入
# import tensorflow as tf

from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)


使用 tensorflow 定义下面的函数。

$$loss = \mathcal{L}(\hat{y}, y) = (\hat y^{(i)} - y^{(i)})^2 \tag{1}$$

In [4]:
y_hat = tf.constant(36, name='y_hat')   # 定义一个 tf 常量 y_hat，值为 36
y = tf.constant(39, name='y')

loss = tf.Variable((y - y_hat)**2, name='loss') # 定义一个 tf 变量，表示上面的 loss 函数

init = tf.global_variables_initializer()        # tf 的固定写法，后面会使用 init 来初始化 loss 变量
with tf.Session() as session:                   # 创建一个 tf 的 session
    session.run(init)                           # 使用这个 session 执行 init 操作
    print(session.run(loss))                    # 使用这个 session 执行 loss 操作，将 loss 值打印

9


编写 tensorflow 程序的一般步骤：

1. 创建变量，在 tensorflow 中有张量 (tensor) 一词
2. 用这些张量来构建每一个操作，如上面的 $(\hat y^{(i)} - y^{(i)})^2$
3. 初始化那些张量。在 tensorflow 中，创建张量时，并没有进行初始化。后面用特殊的语句来初始化这些张量。
4. 创建一个 Session，并且在 Session 中执行操作。
5. 用 Session 来执行前面定义的操作。

In [5]:
# 举例说明 tensorflow 的设计/动工"的特色

a = tf.constant(2)
b = tf.constant(10)
c = tf.multiply(a, b)
print(c)

Tensor("Mul:0", shape=(), dtype=int32)


上述代码执行完时，并没有计算出 c 的值，而是打印出了张量 c 的一些信息。这是因为这些代码只是设计了张量 c，还没有去执行它，所以打印出来的只是一些信息。

为了执行上述的设计，我们需要使用 Session 的 run 方法执行上面的设计，再次打印出张量 c 的值，就得到了我们想要的结果。

我们需要适应 tensorflow 的这种用法：先创建变量和操作，再初始化变量，再创建 Session，再执行操作。

In [6]:
with tf.Session() as session:
    print(session.run(c))

20


### tensorflow 特性：placeholder 占位符

    在 tensorflow 中，我们可以使用占位符来代替一些张量，这些张量在执行操作时，会被替换为真实的值。

    占位符的作用是，在执行操作时，先不管它的值，而是等到真实的值来时，再给它赋值。

In [10]:
# tensorflow placeholrder：占位符

x = tf.placeholder(tf.int64, name='x')              # 创建一个名为 x 的占位符
with tf.Session() as session:
    print(session.run(2 * x, feed_dict={x: 3}))     # 执行时，通过 feed_dict 填充 x 的值，因此执行结果为 6


6


### tensorflow 重要概念：计算图

    当我们创建变量和操作时，仅仅是在 tensorflow 中构建了一个计算图，计算图中可以有占位符。

    这些都只是设计，并没有实际的数据执行。

    知道创建了 session 后，就可以用 session.run 来执行前面设计好的计算图，在执行计算图时，可以往计算图的占位符里填充内容。

    同一个计算图，每次 run 时，都可以往占位符中填充不同的值。

In [11]:
# 使用 tensorflow 实现 Y=WX+b 的线性函数
# 在本例中，我们设 W 的维度是 (4, 3)，X 的维度是 (3, 1)，以及 b 是 (4, 1)

def linear_function():
    np.random.seed(1)

    X = tf.constant(np.random.randn(3, 1), name='X')    # 定义一个维度是 (3, 1) 的常量
    W = tf.constant(np.random.randn(4, 3), name='W')
    b = tf.constant(np.random.randn(4, 1), name='b')
    Y = tf.add(tf.matmul(W, X), b)                      # tf.matmul 会执行矩阵乘法

    # 创建 session
    with tf.Session() as sess:
        result = sess.run(Y)
    
    return result

In [12]:
# 测试

print("result = " + str(linear_function()))

result = [[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]]


In [13]:
# 使用 tensorflow 实现 sigmoid 函数

def sigmoid(z):
    x = tf.placeholder(tf.float32, name='x')    # 定义一个类型为 float32 的占位符

    sigmoid = tf.sigmoid(x)                     # 调用 tensorflow 的 sigmoid 函数，传入 x

    with tf.Session() as sess:
        result = sess.run(sigmoid, feed_dict={x: z})
    
    return result

In [14]:
# 测试

print("sigmoid(0) = " + str(sigmoid(0)))
print("sigmoid(12) = " + str(sigmoid(12)))


sigmoid(0) = 0.5
sigmoid(12) = 0.9999939


### Cost 函数

$$ J = - \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log a^{ [2] (i)} + (1-y^{(i)})\log (1-a^{ [2] (i)} )\large )\small\tag{2}$$

    tensorflow 函数一次性帮我们实现了 sigmoid 和 cost 函数，即如下代码。

```python
tf.nn.sigmoid_cross_entropy_with_logits(logits = ...,  labels = ...)
```

    logits 即最后一层神经元输出的 z，label 即我们的真实标签 y。

    等于一次性实现了下面的函数。

$$- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small\tag{2}$$



In [17]:
# tensorflow cost + sigmoid 函数

def cost(z_in, y_in):
    z = tf.placeholder(tf.float32, name='z')
    y = tf.placeholder(tf.float32, name='y')

    # 使用 sigmoid_cross_entropy_with_logits 来构建 cost 操作
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)

    with tf.Session() as session:
        cost = session.run(cost, feed_dict={z: z_in, y: y_in})
    
    return cost

In [18]:
# test

logits = np.array([0.2, 0.4, 0.7, 0.9])
print('cost = ' + str(cost(logits, np.array([0, 0, 1, 1]))))


cost = [0.79813886 0.91301525 0.40318602 0.34115386]


### One Hot 编码

    在人工智能的编程中，我们经常会遇到多分类问题，softmax就是解决多分类问题的。

    在多分类问题中，我们的 y 向量包含了 0 到 C-1 的数字，里面的 C 表示类别的数目。

    假设 C 是 4，我们有如下的转换：

![](./images/onehot.png)

    右边的向量就是 One Hot 向量，它的每一个元素都是 0，只有一个 1，其他的都是 0。

    如最后一个元素是 1，那么它表示的是第 3 类。倒数第二个元素是 1，那么它表示的是第 2 类。
    
    使用 tensorflow 框架，我们可以使用

```python
tf.one_hot(indices, depth, axis)
```

In [19]:
# tensorflow one_hot

def one_hot_matrix(labels, C_in):
    """
    labels 就是真实标签 y 向量
    C_in 就是类别的数量
    """

    C = tf.constant(C_in, name='C')

    one_hot_matrix = tf.one_hot(indices=labels, depth=C, axis=0)

    with tf.Session() as session:
        one_hot = session.run(one_hot_matrix)
    
    return one_hot

In [20]:
labels = np.array([1, 2, 3, 0, 2, 1])
one_hot = one_hot_matrix(labels, C_in=4)
print("one_hot = " + str(one_hot))


one_hot = [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]


### tensorflow 初始化 0 和 1

 - tf.ones()
 - tf.zeros()

In [21]:
# tensorflow 初始化 1

def ones(shape):
    ones = tf.ones(shape)   # 传入维度

    with tf.Session() as session:
        ones = session.run(ones)
    
    return ones

In [22]:
# test 
print('ones = ' + str(ones([3])))

ones = [1. 1. 1.]
